## pip install

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch torchvision torchaudio

In [3]:
!pip install torchsummary

In [4]:
import torch
import torch.nn as nn
import torchvision
import torchvision.models as models

# @title
import os
from torch.utils.data import Dataset
import numpy as np
import json
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import nn

In [5]:
# prompt: torch seed 고정

import random
import numpy as np
import torch

def same_seeds(seed):
  # Python built-in random module
  random.seed(seed)
  # Numpy
  np.random.seed(seed)
  # Torch
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True

same_seeds(1126)

## customdataset

In [6]:
class VideoClassificationDataset(Dataset):
    def __init__(self, video_dir, label_file, transform=None, fixed_length=30, fixed_channels=3):
        """
        Custom Dataset for video classification.

        Args:
            video_dir (str): 1초짜리 비디오 클립이 저장된 디렉토리 경로.
            label_file (str): 비디오 클립의 라벨이 포함된 JSON 파일 경로.
            transform (callable, optional): 데이터를 전처리하는 함수(예: ToTensor()).
            fixed_length (int, optional): 비디오 클립의 고정된 프레임 수.
            fixed_channels (int, optional): 고정된 채널 수.
        """
        self.video_dir = video_dir
        self.transform = transform
        self.fixed_length = fixed_length
        self.fixed_channels = fixed_channels  # 고정된 채널 수

        # JSON 파일에서 라벨 정보를 로드
        with open(label_file, 'r') as f:
            self.highlight_data = json.load(f)

        # 비디오 파일 이름 목록
        self.video_files = list(self.highlight_data.keys())

    def __len__(self):
        return len(self.video_files)

    def __getitem__(self, idx):
        # 비디오 파일 경로
        video_file = self.video_files[idx]
        prefix = video_file.split('_')[0]  # 파일의 prefix 추출
        video_path = os.path.join(self.video_dir, prefix, f"{video_file}.npz")

        # 비디오 데이터 로드
        try:
            video_data = np.load(video_path)['features']
        except (EOFError, KeyError) as e:
            print(f"Error loading file {video_path}: {e}")
            return None  # None을 반환하여 손상된 파일 건너뛰기

        # 라벨 가져오기
        highlight = self.highlight_data[video_file]['highlight']

        # 필요한 경우, 라벨을 숫자 인덱스로 변환 (0: False, 1: True)
        label = 1 if highlight else 0

        # 필요한 경우, 데이터에 전처리(transform) 적용
        if self.transform:
            video_data = [self.transform(frame) for frame in video_data]
            video_data = torch.stack(video_data)
            if video_data.size(0) < self.fixed_length:
              # (3, 224, 224) 크기의 텐서 생성
              padding_frame = torch.zeros((self.fixed_channels, 224, 224))

              # 부족한 프레임 수 계산
              num_padding_frames = self.fixed_length - video_data.size(0)

              # padding_frame을 num_padding_frames번 쌓기
              padding_frames = [padding_frame] * num_padding_frames
              padding_frames = torch.stack(padding_frames)

              # 기존 video_data와 padding_frames을 결합
              video_data = torch.cat((padding_frames, video_data), dim=0)

        return video_data, label

# Resize transform 정의
resize_transform = transforms.Compose([
    transforms.Lambda(lambda x: x.astype(np.uint8)),  # PIL로 변환할 수 있도록 uint8로 변환
    transforms.ToPILImage(),  # numpy 배열을 PIL 이미지로 변환
    transforms.Resize((224, 224)),  # (H, W) 크기로 변환
    transforms.ToTensor()  # 다시 Tensor로 변환하여 (C, H, W) 형식으로 만듬
])

### true false 비율 1대1로 맞춤

## Dataset 클래스를 인스턴스화

In [7]:
# 예시로 Dataset 클래스를 인스턴스화
train_dataset = VideoClassificationDataset(
    video_dir="/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/train/",
    label_file="/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/datasets/filtered_seconds_result_train_real.json",
    transform=resize_transform,
    fixed_length=30,
    fixed_channels=3
)
test_dataset = VideoClassificationDataset(
    video_dir="/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/test/",
    label_file="/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/datasets/seconds_result_test_real.json",
    transform=resize_transform,
    fixed_length=30,
    fixed_channels=3
)

In [9]:
# 한 개의 데이터 로드 및 크기 확인
video_data, label = test_dataset[2619]
print(f"Resized video data size: {video_data.size()}")
print(f"Label: {label}")

Resized video data size: torch.Size([30, 3, 224, 224])
Label: 0


## model

In [10]:
class BinaryActionRecognitionModel(nn.Module):
    def __init__(self):
        super(BinaryActionRecognitionModel, self).__init__()

        # EfficientNet B0 모델을 가져오고, weights를 사용하여 미리 학습된 가중치 로드
        self.efficientnet = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
        self.efficientnet = nn.Sequential(*list(self.efficientnet.children())[:-1])  # Feature extractor로 사용

        # Classification Head
        self.fc1 = nn.Linear(1280, 64)  # EfficientNet B0의 출력 크기에 맞게 설정
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        batch_size, time_steps, c, h, w = x.size()
        # Rescale input
        x = x * 255.0  # Assuming input is in range [0, 1]
        # Reshape input for EfficientNet
        x = x.view(batch_size * time_steps, c, h, w)
        x = self.efficientnet(x)
        x = x.view(batch_size, time_steps, -1)

        # Classification head
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)

        # Global Average Pooling (along time dimension)
        x = x.mean(dim=1)  # GlobalAveragePooling3D의 역할을 하는 부분
        x = x[:, 1]
        return x

# 모델 인스턴스 생성
clip_size = (28, 3, 320, 640)  # 예시 입력 크기 (time_steps, channels, height, width)
model = BinaryActionRecognitionModel()

# 모델의 입력 텐서 테스트 (batch_size, time_steps, channels, height, width)
input_tensor = torch.randn(2, *clip_size)  # batch_size=8, clip_size=(30, 3, 320, 640)
output = model(input_tensor)
print(output)
print(output.shape)  # Expected output: (8, 2)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 60.6MB/s]


tensor([0.4916, 0.4956], grad_fn=<SelectBackward0>)
torch.Size([2])


In [ ]:
from torchsummary import summary

# 모델을 GPU로 옮김 (가능한 경우)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BinaryActionRecognitionModel().to(device)

# Generate Summary of the Model
summary(model, input_size = (30, 3, 224, 224), device=str(device))  # 입력 크기 (time_steps, channels, height, width)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              SiLU-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              SiLU-6         [-1, 32, 112, 112]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
              SiLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
SqueezeExcitation-12         [-1, 32, 112, 112]               0
           Conv2d-13         [-1, 16, 112, 112]             512
      BatchNorm2d-14         [-1, 16, 1

In [11]:
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/checkpoint/epoch0_step16100.pth"

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm  # tqdm 모듈 임포트

# 1. DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

# 2. 모델 초기화
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = BinaryActionRecognitionModel().to(device)

# 3. 손실 함수 및 옵티마이저 정의
criterion = nn.BCELoss()  # 이진 분류 문제에 적합한 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 저장된 체크포인트 불러오기
start_epoch = 0
start_step = 0
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/checkpoint/epoch0_step5200.pth"


try:
    checkpoint = torch.load(checkpoint_path)
    # checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    start_step = checkpoint.get('step', 0)  # step 정보 로드 (기본값 0)
    running_loss = checkpoint['loss']
    print(f"Checkpoint loaded. Resuming training from epoch {start_epoch+1}, step {start_step}.")
except FileNotFoundError:
    print("No checkpoint found. Starting from scratch.")
    running_loss = 0.0

# 4. 모델 학습 루프
num_epochs = 2  # 학습할 에포크 수

try:
    for epoch in range(start_epoch, num_epochs):
        model.train()  # 모델을 학습 모드로 설정
        running_loss = 0.0

        # train_loader의 iterator 생성 (특정 step부터 시작하기 위해)
        train_loader_iterator = iter(train_loader)

        # 기존 step을 건너뛰기 위해 iterator를 advance
        for _ in tqdm(range(start_step)):
            next(train_loader_iterator, None)

        # tqdm으로 진행률 표시
        for i, (inputs, labels) in enumerate(tqdm(train_loader_iterator, desc=f"Epoch {epoch+1}/{num_epochs}", initial=start_step), start=start_step + 1):
            try:
                # 데이터를 GPU로 이동
                inputs, labels = inputs.to(device), labels.to(device)
                if inputs.size() != (inputs.size(0), 30, 3, 224, 224):
                    print("Input size wrong:", inputs.size())
                    continue  # 크기가 올바르지 않으면 다음 배치로 넘어감

                # 옵티마이저 초기화
                optimizer.zero_grad()

                # 모델에 입력을 전달하여 출력을 얻음
                outputs = model(inputs)
                # 손실 계산
                loss = criterion(outputs, labels.float())

                # 역전파 및 옵티마이저 스텝
                loss.backward()
                optimizer.step()

                # 손실 누적
                running_loss += loss.item()

                # 100 스텝마다 손실 출력
                if i % 100 == 0:
                    print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i}/{len(train_loader)}], Loss: {running_loss / i:.4f}")

                    # 100 스텝마다 체크포인트 저장
                    torch.save({
                        'epoch': epoch,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': running_loss,
                        'step': i,
                    }, f"/content/drive/MyDrive/AIKU_summer/checkpoint/epoch{epoch}_step{i}.pth")
                    print(f"Checkpoint saved at epoch {epoch+1}, step {i}.")

            except Exception as e:
                print(f"Error at step {i} during epoch {epoch+1}: {e}")
                continue  # 오류가 발생하면 다음 배치로 넘어감

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

        # 에포크 종료 후 체크포인트 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': running_loss,
            'step': 0,  # 다음 epoch에서는 step을 0부터 시작
        }, f"/content/drive/MyDrive/AIKU_summer/checkpoint/epoch{epoch}_fin.pth")
        print(f"Checkpoint saved at epoch {epoch+1}.")
        start_step = 0  # 다음 epoch 시작 시 step을 초기화

except KeyboardInterrupt:
    # 학습이 중단되었을 때 현재 상태를 저장
    print("Training interrupted. Saving checkpoint...")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': running_loss,
        'step': start_step,  # 중단 시의 step 저장
    }, f"/content/drive/MyDrive/AIKU_summer/checkpoint/epoch{epoch}_interrupted.pth")
    print(f"Checkpoint saved at epoch {epoch+1}.")
    print("Training has been stopped manually.")

cuda


<ipython-input-13-0714001fda5f>:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


Checkpoint loaded. Resuming training from epoch 1, step 5200.


  3%|▎         | 175/5200 [16:51<8:03:52,  5.78s/it]

Training interrupted. Saving checkpoint...


RuntimeError: Parent directory /content/drive/MyDrive/AIKU_summer/checkpoint does not exist.

In [ ]:

# Checkpoint 불러오기 코드
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/checkpoint/epoch0_step16100.pth"
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch']
running_loss = checkpoint['loss']
print(f"Checkpoint loaded. Resuming training from epoch {start_epoch+1}.")

# 학습 재개
num_epochs = 2  # 추가 학습할 에포크 수
total_epochs = start_epoch + num_epochs  # 전체 학습 에포크 계산

# 모델을 학습 모드로 전환
model.train()

for epoch in range(start_epoch + 1, total_epochs + 1):
    epoch_loss = 0.0

    for i, (inputs, labels) in enumerate(train_loader, 0):  # train_loader는 학습 데이터 로더입니다.
        inputs, labels = inputs.to(device), labels.to(device)  # GPU로 데이터를 이동 (필요 시)

        optimizer.zero_grad()  # 이전 미분 값 초기화

        outputs = model(inputs)  # 모델 출력 계산
        loss = criterion(outputs, labels)  # 손실 계산

        loss.backward()  # 역전파 수행
        optimizer.step()  # 가중치 업데이트

        epoch_loss += loss.item()  # 현재 에포크의 손실 누적

        if i % 100 == 99:  # 100번째 배치마다 진행 상황 출력
            print(f"[Epoch {epoch}, Batch {i+1}] Loss: {epoch_loss / 100:.4f}")
            epoch_loss = 0.0

    # 에포크 완료 후, 체크포인트 저장
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss,
    }, checkpoint_path)

    print(f"Epoch {epoch} completed and checkpoint saved.")

print("Training resumed and completed.")

In [ ]:
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/checkpoint/epoch0_step16100.pth"
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])

In [14]:
!pip install scikit-learn


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
from tqdm import tqdm  # tqdm 모듈 임포트

test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)
# 5. 검증 단계 (옵션)
# 모델 초기화 및 체크포인트 불러오기
checkpoint_model = "/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/checkpoint/epoch0_step16100.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 초기화 (모델 정의는 이미 있다고 가정)
model = BinaryActionRecognitionModel()

# 체크포인트에서 모델 가중치 불러오기
checkpoint = torch.load(checkpoint_model, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

# 모델을 평가 모드로 설정
model.eval()

# 모델을 GPU로 이동 (가능한 경우)
model.to(device)
print(device)
# 결과를 저장할 딕셔너리 초기화
output_dict = {}
# 변수 초기화
correct = 0
total = 0
all_labels = []
all_predictions = []
all_outputs = []  # 모델 출력을 저장할 리스트
batch_idx = 0

with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Processing test data"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()  # 임계값 0.5를 기준으로 예측
        correct += (predicted == labels).sum().item()
        total += labels.numel()

        # outputs와 labels 값을 리스트에 추가
        all_outputs.extend(outputs.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        # 100개마다 결과 출력
        if batch_idx % 100 == 0:
            print(f"Batch {batch_idx}: Correct: {correct}, Total: {total}")
        batch_idx += 1

<ipython-input-16-985497c945c3>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_model, map_location=device)


cuda


Processing test data:   0%|          | 1/22653 [00:01<8:56:17,  1.42s/it]

Batch 0: Correct: 2, Total: 2


Processing test data:   0%|          | 20/22653 [00:54<17:05:20,  2.72s/it]


BadZipFile: File is not a zip file